# Conway's Game of Life (with Boundaries Tracking)
Conway's Game of Life is a cellular automata simulation that follows simple rules to create visual patterns.
The *game* is played on a two-dimensional board (a grid) of cells. Each cell can be either black or white.
The board evolves according to the following rules:
- Living (black) cells with two or three neighbors stay alive in the next step of the simulation;
- Dead (white) cells with exactly three living neighbors become alive in the next step of the simulation;
- Any other cell dies or stays dead in the next step of the simulation.

The living or dead state of the cells in the next step of the simulation depends entirely on their current state. There is no *memory* whatsoever for the grid cells beside the current board status, which rules the living or dead state of the cells in the next step of the simulation.

The following implementation is particularly efficient when the number of living cells is not too large. Instead of updating the whole grid at each time step, I keep track of the *boundary* of the *alive set*. Actually, the *t+1* alive set can be obtained simply looking at the time *t* alive set and its boundary. Have a look!

In [ ]:
import copy, random, sys, time, math
import numpy as np
import pygame

GRID_HEIGHT = 100
GRID_WIDTH = 100

FRAME_RATE = 4

CELL_SIZE = 800/max(GRID_HEIGHT, GRID_WIDTH)

WINDOW_HEIGHT = GRID_HEIGHT * CELL_SIZE
WINDOW_WIDTH = GRID_WIDTH * CELL_SIZE

DEAD_COLOR = (255,255,255)
ALIVE_COLOR = (0,0,0)

def random_config():
    alive_set = set()
    
    for y in range(GRID_HEIGHT):
        for x in range(GRID_WIDTH):
            if random.randint(0,1) == 1:
                alive_set.add((x, y))
    
    return alive_set


def pulsars():
    c0 = GRID_WIDTH // 2
    c1 = GRID_HEIGHT // 2
    
    return set([(c0, c1-2), (c0, c1-3), (c0, c1-4), (c0, c1+3), (c0, c1+4), (c0, c1+5), (c0+1, c1-1), (c0+1, c1-2), (c0+1, c1+2), (c0+1, c1+3), (c0+2, c1-1), (c0+2, c1+2), (c0+3, c1-1), (c0+3, c1+2), (c0-3, c1-2), (c0-3, c1-3), (c0-3, c1-4), (c0-3, c1+3), (c0-3, c1+4), (c0-3, c1+5), (c0-4, c1-1), (c0-4, c1-2), (c0-4, c1+2), (c0-4, c1+3), (c0-5, c1-1), (c0-5, c1+2), (c0-6, c1-1), (c0-6, c1+2), (c0-9, c1-1), (c0-9, c1-2), (c0-9, c1-3), (c0-9, c1+3), (c0-9, c1+4), (c0-9, c1+5), (c0-9, c1-1), (c0-9, c1-2), (c0-9, c1-3), (c0-9, c1+5), (c0-9, c1+3), (c0-9, c1+4), (c0-14, c1-1), (c0-14, c1-2), (c0-14, c1-3), (c0-14, c1+5), (c0-14, c1+3), (c0-14, c1+4), (c0-16, c1-1), (c0-16, c1-2), (c0-16, c1-3), (c0-16, c1+5), (c0-16, c1+3), (c0-16, c1+4), (c0-21, c1-1), (c0-21, c1-2), (c0-21, c1-3), (c0-21, c1+5), (c0-21, c1+3), (c0-21, c1+4), (c0-11, c1), (c0-12, c1), (c0-13, c1), (c0-17, c1), (c0-18, c1), (c0-19, c1), (c0-11, c1+2), (c0-12, c1+2), (c0-13, c1+2), (c0-17, c1+2), (c0-18, c1+2), (c0-19, c1+2), (c0-11, c1+7), (c0-12, c1+7), (c0-13, c1+7), (c0-17, c1+7), (c0-18, c1+7), (c0-19, c1+7), (c0-11, c1-5), (c0-12, c1-5), (c0-13, c1-5), (c0-17, c1-5), (c0-18, c1-5), (c0-19, c1-5), (c0+6, c1-1), (c0+7, c1-1), (c0+8, c1-1), (c0+9, c1-1), (c0+10, c1-1), (c0+16, c1-1), (c0+17, c1-1), (c0+18, c1-1), (c0+19, c1-1), (c0+20, c1-1), (c0+6, c1+3), (c0+7, c1+3), (c0+8, c1+3), (c0+9, c1+3), (c0+10, c1+3), (c0+16, c1+3), (c0+17, c1+3), (c0+18, c1+3), (c0+19, c1+3), (c0+20, c1+3), (c0+11, c1-6), (c0+11, c1-5), (c0+11, c1-4), (c0+11, c1-3), (c0+11, c1-2), (c0+11, c1+4), (c0+11, c1+5), (c0+11, c1+6), (c0+11, c1+7), (c0+11, c1+8), (c0+15, c1-6), (c0+15, c1-5), (c0+15, c1-4), (c0+15, c1-3), (c0+15, c1-2), (c0+15, c1+4), (c0+15, c1+5), (c0+15, c1+6), (c0+15, c1+7), (c0+15, c1+8), (c0+9, c1-2), (c0+9, c1+4), (c0+10, c1-3), (c0+10, c1+5), (c0+17, c1-2), (c0+17, c1+4), (c0+16, c1-3), (c0+16, c1+5)])


def gosper_glider_gun():
    c0 = GRID_WIDTH // 2
    c1 = GRID_HEIGHT // 2
    
    return set([(c0, c1), (c0, c1-1), (c0, c1-2), (c0+1, c1), (c0+1, c1-1), (c0+1, c1-2), (c0+2, c1-3), (c0+2, c1+1), (c0+4, c1-3), (c0+4, c1+1), (c0+4, c1-4), (c0+4, c1+2), (c0+14, c1-1), (c0+14, c1-2), (c0+15, c1-1), (c0+15, c1-2), (c0-3, c1+1), (c0-4, c1+1), (c0-4, c1), (c0-4, c1+2), (c0-5, c1-1), (c0-5, c1+3), (c0-6, c1+1), (c0-7, c1-2), (c0-7, c1+4), (c0-8, c1-2), (c0-8, c1+4), (c0-9, c1-1), (c0-9, c1+3), (c0-10, c1), (c0-10, c1+1), (c0-10, c1+2), (c0-19, c1), (c0-20, c1), (c0-19, c1+1), (c0-20, c1+1)])


def PBC(coord_0,coord_1):
    if coord_0 < 0 or coord_0 >= GRID_WIDTH:
        coord_0 = coord_0 % GRID_WIDTH
    if coord_1 < 0 or coord_1 >= GRID_HEIGHT:
        coord_1 = coord_1 % GRID_HEIGHT
    return (coord_0,coord_1)


def contour(coord):
    contour = set()
    contour.add(PBC(coord[0]-1, coord[1]+1))
    contour.add(PBC(coord[0], coord[1]+1))
    contour.add(PBC(coord[0]+1, coord[1]+1))
    contour.add(PBC(coord[0]+1, coord[1]))
    contour.add(PBC(coord[0]+1, coord[1]-1))
    contour.add(PBC(coord[0], coord[1]-1))
    contour.add(PBC(coord[0]-1, coord[1]-1))
    contour.add(PBC(coord[0]-1, coord[1]))
    return contour


def boundary(alive_set):
    boundary_set = set()
    for cell_coord in alive_set:
        boundary_set = boundary_set.union(contour(cell_coord))
    boundary_set -= alive_set
    return boundary_set


def contour_alive_sum(coord, alive_set):
    count = 0
    for c in contour(coord):
        if c in alive_set:
            count += 1
    return count


def evolution(alive_set, boundary_set):
    alive_set_next = set()
    for cell in alive_set:
        if 2 <= contour_alive_sum(cell, alive_set) <= 3:
            alive_set_next.add(cell)
    for void in boundary_set:
        if contour_alive_sum(void, alive_set) == 3:
            alive_set_next.add(void)
    return(alive_set_next)


def drawGrid(window, alive_set):
    for x in range(GRID_WIDTH):
        for y in range(GRID_HEIGHT):
            rect = pygame.Rect(x*CELL_SIZE, y*CELL_SIZE, CELL_SIZE, CELL_SIZE)
            if (x,y) in alive_set:
                pygame.draw.rect(window, ALIVE_COLOR, rect)
            else:
                pygame.draw.rect(window, DEAD_COLOR, rect)

                
def main():
    
    starting_conf = input("Select the starting configuration: \n - 1: random\n - 2: pulsars\n - 3: Gosper glider gun\n")
    if starting_conf == "1":
        alive_set = random_config()
    elif starting_conf == "2":
        alive_set = pulsars()
    elif starting_conf == "3":
        alive_set = gosper_glider_gun()
    
    boundary_set = boundary(alive_set)
    
    pygame.init()
    
    window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
    pygame.display.set_caption("Conway's game of life")
    window.fill(ALIVE_COLOR)
    pygame.display.flip()
    drawGrid(window, alive_set)
    pygame.display.flip()
    
    clock = pygame.time.Clock()
    run = True
    pause = True
    
    while True:
        
        for event in pygame.event.get():
            
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                break

            elif event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                column = math.floor(pos[0] / CELL_SIZE)
                row = math.floor(pos[1] / CELL_SIZE)
                
                coord_click = (column, row)
                
                if coord_click in alive_set:
                    alive_set -= set([coord_click])
                else:
                    alive_set = alive_set.union(set([coord_click]))
                
                drawGrid(window, alive_set)
                pygame.display.flip()
                clock.tick(FRAME_RATE)
            
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    pause = not pause
        
        if not run:
            break
        
        if pause:
            continue
     
        alive_set = evolution(alive_set, boundary_set)
        boundary_set = boundary(alive_set)
        
        drawGrid(window, alive_set)
        pygame.display.flip()
        
        clock.tick(FRAME_RATE)
    
    pass

if __name__ == '__main__':
    main()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


Select the starting configuration: 
 - 1: random
 - 2: pulsars
 - 3: Gosper glider gun
 2
